In [ ]:
import Data.Maybe (fromMaybe)

In [ ]:
data BTree a = Empty 
    | Leaf a 
    | Branch (BTree a) a (BTree a)
    deriving (Eq, Show)

In [ ]:
(|>) = flip ($)

In [ ]:
contains :: (Ord a) => a -> BTree a -> Bool

contains _ Empty = False
contains x (Leaf v) = x == v

contains x (Branch l v r)
    | x == v = True
    | x < v  = contains x l
    | otherwise  = contains x r

In [ ]:
insert :: (Ord a) => a -> BTree a -> BTree a

insert x btree
    | contains x btree = btree
    | otherwise = insert' x btree
        where
            insert' x Empty = Leaf x

            insert' x (Leaf v)
                | x < v = Branch (Leaf x) v Empty
                | otherwise = Branch Empty v (Leaf x)

            insert' x (Branch l v r)
                | x < v = Branch (insert' x l) v r
                | otherwise = Branch l v (insert' x r)

In [ ]:
findMin :: (Ord a) => BTree a -> Maybe a

findMin bt = case bt of
    Empty -> Nothing
    Leaf v -> Just v
    Branch l v _ -> case l of
        Empty -> Just v
        otherwise -> findMin l

In [ ]:
delete :: (Ord a) => a -> BTree a -> BTree a

delete x bt = case bt of 
    Empty -> Empty
    Leaf v -> if x == v then Empty else Leaf v
    Branch l v r -> 
        case compare x v of
            LT -> Branch (delete x l) v r
            GT -> Branch l v (delete x r)
            EQ -> case minRightBranchValue of
                      Just v' -> Branch l v' (delete v' r)
                      Nothing -> l -- right branch is empty
                  where
                      minRightBranchValue = findMin r

In [ ]:
instance Foldable BTree where
    foldr f z Empty = z
    foldr f z (Leaf x) = f x z
    foldr f z (Branch l v r) = foldr f (f v (foldr f z r)) l

In [ ]:
toBTree :: (Foldable f, Ord a) => f a -> BTree a

toBTree = foldr insert Empty

In [ ]:
toList = foldr (:) []

In [ ]:
[1, 2, 0, 1, 1, 1, 2, 3, -1, 5, -2, 4] 
    |> reverse
    |> toBTree
    |> insert 1 
    |> insert 2 
    |> insert 0 
    |> insert 3
    |> delete 1
    |> delete 2
    |> delete 3
    |> delete 0
    |> toList